In [1]:
import datetime as dt
from lxml import html
from bs4 import BeautifulSoup
import urllib
import sys
import requests
import operator
from datetime import timedelta
import csv
import pandas as pd
import numpy as np
import math

In [2]:
dt.date.today() == dt.date(2020, 2, 6)

False

In [3]:
def fill_games_list(games_table,game_list,yesterday):
    for x in games_table:

        if x.find("tbody").findAll("tr")[0].find("a").has_attr('href') and x.find("tbody").findAll("tr")[1].find("a").has_attr('href'):
            
            winner = x.find("tbody").find("tr",{"class":"winner"}).find("a").text
            winner_score = x.find("tbody").find("tr",{"class":"winner"}).find("td",{"class":"right"}).text
            loser = x.find("tbody").find("tr",{"class":"loser"}).find("a").text
            loser_score = x.find("tbody").find("tr",{"class":"loser"}).find("td",{"class":"right"}).text
            home_team =x.find("tbody").findAll("tr")[1].find("a").text
            away_team =x.find("tbody").findAll("tr")[0].find("a").text
            
            for x in [winner, loser, home_team, away_team]:
                if x in ['UCSB',"St. Peter's","St. Joseph's","BYU","Saint Mary's",'UMKC','SIU-Edwardsville','UTSA','UT-Martin','UTEP','LSU','California','Penn','USC','Southern Miss','LIU-Brooklyn','TCU','ETSU','Central Connecticut','Pitt','Detroit','Ole Miss','UIC','UNC','NC State','USC Upstate','UMBC','UMass','UMass-Lowell','UNC Asheville','UConn','VCU','SMU','UCF','UNLV','UNC Wilmington','UNC Greensboro']:
                    x = x.replace("UNC Asheville", "North Carolina-Asheville")
                    x = x.replace("UNC Wilmington", "North Carolina-Wilmington")
                    x = x.replace("UNC Greensboro", "North Carolina-Greensboro")
                    x = x.replace("UNC", "North Carolina")
                    x = x.replace("NC State", "North Carolina State")
                    x = x.replace("USC Upstate", "South Carolina Upstate")
                    x = x.replace("UMBC", "Maryland-Baltimore County")
                    x = x.replace("UMass", "Massachusetts")
                    x = x.replace("UConn", "Connecticut")
                    x = x.replace("VCU", "Virginia Commonwealth")
                    x = x.replace("SMU", "Southern Methodist")
                    x = x.replace("UCF", "Central Florida")
                    x = x.replace("UNLV", "Nevada-Las Vegas")
                    x = x.replace("UIC", "Illinois-Chicago")
                    x = x.replace("Ole Miss", "Mississippi")
                    x = x.replace("Detroit", "Detroit Mercy")
                    x = x.replace("Pitt", "Pittsburgh")
                    x = x.replace("Central Connecticut", "Central Connecticut State")
                    x = x.replace("ETSU", "East Tennessee State")
                    x = x.replace("TCU", "Texas Christian")
                    x = x.replace("LIU-Brooklyn", "Long Island University")
                    x = x.replace("Southern Miss", "Southern Mississippi")
                    x = x.replace("California", "University of California")
                    x = x.replace("USC", "Southern California")
                    x = x.replace("Penn", "Pennsylvania")
                    x = x.replace("LSU", "Louisiana State")
                    x = x.replace("UTEP", "Texas-El Paso")
                    x = x.replace("UT-Martin", "Tennessee-Martin")
                    x = x.replace("UTSA", "Texas-San Antonio")
                    x = x.replace("SIU-Edwardsville", "SIU Edwardsville")
                    x = x.replace("UMKC", "Missouri-Kansas City")
                    x = x.replace("Saint Mary's", "Saint Mary's (CA)")
                    x = x.replace("BYU", "Brigham Young")
                    x = x.replace("St. Joseph's", "Saint Joseph's")
                    x = x.replace("St. Peter's", "Saint Peter's")
                    x = x.replace("UCSB", "UC-Santa Barbara")
                
            game_list.append((yesterday,winner,winner_score,loser,loser_score,home_team,away_team))
        else:
            pass

In [4]:
def get_yest_games():
    game_list = []
    for x in range(180,0,-1):
        yesterday = dt.date.today()- timedelta(days=x)
        if yesterday == dt.date(2020, 1, 20):
            continue
        SR_games = "https://www.sports-reference.com/cbb/boxscores/index.cgi?month="+str(yesterday.month)+"&day="+str(yesterday.day)+"&year="+str(yesterday.year)
        #print(SR_games)
        try:
            games = requests.get(SR_games)
            soup_games = BeautifulSoup(games.content, 'lxml')
            games_table=soup_games.findAll("div",{"class":"game_summary nohover"})
            print(games_table)
            fill_games_list(games_table,game_list,yesterday)
        except:
            print("ERROR!")
    return game_list

In [5]:
def add_ken_pom(teams_dict, KP):

    kenpom = requests.get(KP)
    soup_kenpom = BeautifulSoup(kenpom.content, 'lxml')
    kenpom_table=soup_kenpom.find("table",{"id":"ratings-table"})
    kenpom_table.findAll("tr")[2].findAll("td")[4]

    for x in kenpom_table.findAll("tr"):
        if x.find("a") == None or x.find("td") == None:
            pass
        else:
            team = x.find("a").text
            rank = int(x.find("td").text)
            val = x.findAll("td")[4].text
            if "Merrimack" in team:
              continue             
            teams_dict[team] = {}                       
            teams_dict[team]['Team'] = [team]
            teams_dict[team]['KenPomRank'] = rank
            teams_dict[team]['KenPomVal'] = val

# # This section adds BPI to the teams_dict 

def add_BPI(teams_dict, BP):
    for link in BP:
        r = requests.get(link)
        soup2 = BeautifulSoup(r.content, 'lxml')
        table=soup2.find("table",{"class":"bpi__table"})
        if table is None:
            continue
        for x in table.find("tbody").findAll("tr"):
            if x.find("a") == None or x.find("td") == None:
              pass
            else:
                team = x.find("span",{"class":"team-names"}).text
                BPI_name = x.find("span",{"class":"team-names"}).text
                rank = x.findAll("td")[0].text
                val = x.findAll("td")[6].text
                if "Long Island University" in team:
                    team = "LIU"
                if "Loyola (Chi)" in team:
                    team = "Loyola Chicago"
                if "Loyola (MD)" in team:
                    team = "Loyola MD"

                if "Youngstown St" in team:
                    team = "Youngstown St."
                if "Savannah St" in team:
                  continue
                if "Merrimack" in team:
                  continue
                if "Umbc" in team:
                    team = "UMBC"
                if "-" in team or "(" in team or ")" in team:
                    team = team.replace("-", " ")
                    team = team.replace("(", "")
                    team = team.replace(")", "")
                if "NC St" in team: 
                    team = team.replace("NC", "North Carolina")
                if "State" in team:
                    team = team.replace("State", "St.")
                if "CSU" in team:
                    team = team.replace("CSU", "Cal St.")
                if "CC" in team:
                    team = team.replace("CC", "Corpus Chris")
                if "Se " in team or "SE " in team or "Mt." in team:
                    team = team.replace("Se", "Southeast")
                    team = team.replace("SE", "Southeastern")
                    team = team.replace("Mt.", "Mount")
                if team in ["North Carolina St.", "Southeastern Missouri St","San José St","South Carolina Upstate","California Baptist", "Detroit Mercy", "McNeese", "Southeast Missouri St", "UT Martin", "Nicholls", "Grambling", "Miami", "Ole Miss", "UConn", "Pennsylvania", "UMass", "UL Monroe", "Omaha", "UIC", "Florida International", "St. Francis BKN", "Hawai'i"]:
                    team = team.replace("North Carolina St.", "N.C. State")
                    team = team.replace("San José St", "San Jose St.")
                    team = team.replace("South Carolina Upstate", "USC Upstate")
                    team = team.replace("California Baptist", "Cal Baptist")
                    team = team.replace("Detroit Mercy", "Detroit")
                    team = team.replace("McNeese", "McNeese St.")
                    team = team.replace("Southeast Missouri St", "Southeast Missouri St.")
                    team = team.replace("Southeastern Missouri St", "Southeast Missouri St.")
                    team = team.replace("UT Martin", "Tennessee Martin")
                    team = team.replace("Nicholls", "Nicholls St.")
                    team = team.replace("Grambling", "Grambling St.")
                    team = team.replace("Miami", "Miami FL")
                    team = team.replace("Ole Miss", "Mississippi")
                    team = team.replace("UConn", "Connecticut")
                    team = team.replace("Pennsylvania", "Penn")
                    team = team.replace("UMass", "Massachusetts")
                    team = team.replace("UL Monroe", "Louisiana Monroe")
                    team = team.replace("Omaha", "Nebraska Omaha")
                    team = team.replace("UIC", "Illinois Chicago")
                    team = team.replace("Florida International", "FIU")
                    team = team.replace("St. Francis BKN", "St. Francis NY")
                    team = team.replace("Hawai'i", "Hawaii")


                teams_dict[team]['BPIrank'] = rank
                teams_dict[team]['BPIval'] = val
                teams_dict[team]['Team'].append(BPI_name)

# #  This Section brings in SRS to teams

def add_sports_ref(teams_dict, SRS):
    srs = requests.get(SRS)
    soup_srs = BeautifulSoup(srs.content, 'lxml')
    srs_table=soup_srs.find("table",{"id":"ratings"})

    for x in srs_table.find("tbody").findAll("tr"):
        if x.find("a") == None or x.findAll("td")[13] == None:
            pass
        else:
            team = x.find("a").text
            SRS_name = x.find("a").text
            srs_val = x.findAll("td")[14].text
            o_eff = x.findAll("td")[15].text
            d_eff = x.findAll("td")[16].text
            ppg = x.findAll("td")[6].text
            papg = x.findAll("td")[7].text
            if "Savannah St" in team:
              continue
            if "Merrimack" in team:
              continue            
            if "State" in team:
                team = team.replace("State", "St.")
            if "University" in team:
                team = team.replace("University ", "")
            if "of " in team:
                team = team.replace("of ", "")
            if "-" in team or "(" in team or ")" in team:
                team = team.replace("-", " ")
                team = team.replace("(", "")
                team = team.replace(")", "")
            if team in ["Cal St. Long Beach","North Carolina St.","North Carolina Asheville","South Carolina Upstate","Albany NY","Prairie View","Missouri Kansas City","Long Island University","Massachusetts Lowell","Maryland Baltimore County","North Carolina Wilmington","Saint Francis PA","Grambling","Central Connecticut St.","Detroit Mercy","Texas A&M Corpus Christi","Omaha","Texas Arlington","Florida International","California Baptist","Bowling Green St.","Texas San Antonio","Texas Rio Grande Valley","Texas El Paso","Alabama Birmingham","Southern Mississippi","Citadel","North Carolina Greensboro","Southern Methodist","Southern California","Pennsylvania","Nevada Las Vegas","College Charleston","St. John's NY","Loyola IL","Brigham Young","Virginia Commonwealth","Central Florida","Louisiana St.", "Texas Christian", "Saint Mary's CA"]:
                team = team.replace("Cal St. Long Beach", "Long Beach St.")
                team = team.replace("North Carolina St.", "N.C. State")                
                team = team.replace("North Carolina Asheville", "UNC Asheville")
                team = team.replace("South Carolina Upstate", "USC Upstate")
                team = team.replace("Albany NY", "Albany")
                team = team.replace("Prairie View", "Prairie View A&M")
                team = team.replace("Missouri Kansas City", "UMKC")
                team = team.replace("Long Island University", "LIU")
                team = team.replace("Massachusetts Lowell", "UMass Lowell")
                team = team.replace("Maryland Baltimore County", "UMBC")
                team = team.replace("North Carolina Wilmington", "UNC Wilmington")
                team = team.replace("Saint Francis PA", "St. Francis PA")
                team = team.replace("Grambling", "Grambling St.")
                team = team.replace("Central Connecticut St.", "Central Connecticut")
                team = team.replace("Detroit Mercy", "Detroit")
                team = team.replace("Texas A&M Corpus Christi", "Texas A&M Corpus Chris")
                team = team.replace("Omaha", "Nebraska Omaha")
                team = team.replace("Texas Arlington", "UT Arlington")
                team = team.replace("Florida International", "FIU")
                team = team.replace("California Baptist", "Cal Baptist")
                team = team.replace("Bowling Green St.", "Bowling Green")
                team = team.replace("Texas San Antonio", "UTSA")
                team = team.replace("Texas Rio Grande Valley", "UT Rio Grande Valley")
                team = team.replace("Texas El Paso", "UTEP")
                team = team.replace("Alabama Birmingham", "UAB")
                team = team.replace("Southern Mississippi", "Southern Miss")
                team = team.replace("Citadel", "The Citadel")
                team = team.replace("North Carolina Greensboro", "UNC Greensboro")
                team = team.replace("Southern Methodist", "SMU")
                team = team.replace("Southern California", "USC")
                team = team.replace("Pennsylvania", "Penn")
                team = team.replace("Nevada Las Vegas", "UNLV")
                team = team.replace("College Charleston", "Charleston")
                team = team.replace("St. John's NY", "St. John's")
                team = team.replace("Loyola IL", "Loyola Chicago")
                team = team.replace("Brigham Young", "BYU")
                team = team.replace("Virginia Commonwealth", "VCU")
                team = team.replace("Central Florida", "UCF")
                team = team.replace("Louisiana St.", "LSU")
                team = team.replace("Texas Christian", "TCU")
                team = team.replace("Saint Mary's CA", "Saint Mary's")

            teams_dict[team]['SRS'] = srs_val
            teams_dict[team]['o_eff'] = o_eff
            teams_dict[team]['d_eff'] = d_eff
            teams_dict[team]['ppg'] = ppg
            teams_dict[team]['papg'] = papg
            teams_dict[team]['Team'] =SRS_name

def add_advanced(teams_dict):
    SR_ratings_team = "https://www.sports-reference.com/cbb/seasons/2020-advanced-school-stats.html"
    SR_ratings_opp = "https://www.sports-reference.com/cbb/seasons/2020-advanced-opponent-stats.html"
    
    team_ratings = requests.get(SR_ratings_team)
    team_soup_ratings = BeautifulSoup(team_ratings.content, 'lxml')
    team_ratings_table=team_soup_ratings.find("table",{"id":"adv_school_stats"})
    teams = team_ratings_table.find("tbody").findAll("tr")
    
    opp_ratings = requests.get(SR_ratings_opp)
    opp_soup_ratings = BeautifulSoup(opp_ratings.content, 'lxml')
    opp_ratings_table=opp_soup_ratings.find("table",{"id":"adv_opp_stats"})
    opps = opp_ratings_table.find("tbody").findAll("tr")
    
    for x in teams:
        if x.find("a") == None or x.findAll("td")[16] == None:
            pass
        else:
            team = x.find('a').text
            pace = x.findAll('td')[16].text
            Three_Par = x.findAll('td')[19].text
            TS = x.findAll('td')[20].text
            Tov = x.findAll('td')[26].text
            if "Savannah St" in team:
              continue
            if "Merrimack" in team:
              continue            
            if "State" in team:
                team = team.replace("State", "St.")
            if "University" in team:
                team = team.replace("University ", "")
            if "of " in team:
                team = team.replace("of ", "")
            if "-" in team or "(" in team or ")" in team:
                team = team.replace("-", " ")
                team = team.replace("(", "")
                team = team.replace(")", "")
            if team in ["North Carolina St.", "Cal St. Long Beach", "North Carolina Asheville","South Carolina Upstate","Albany NY","Prairie View","Missouri Kansas City","Long Island University","Massachusetts Lowell","Maryland Baltimore County","North Carolina Wilmington","Saint Francis PA","Grambling","Central Connecticut St.","Detroit Mercy","Texas A&M Corpus Christi","Omaha","Texas Arlington","Florida International","California Baptist","Bowling Green St.","Texas San Antonio","Texas Rio Grande Valley","Texas El Paso","Alabama Birmingham","Southern Mississippi","Citadel","North Carolina Greensboro","Southern Methodist","Southern California","Pennsylvania","Nevada Las Vegas","College Charleston","St. John's NY","Loyola IL","Brigham Young","Virginia Commonwealth","Central Florida","Louisiana St.", "Texas Christian", "Saint Mary's CA"]:
                team = team.replace("Cal St. Long Beach", "Long Beach St.")
                team = team.replace("North Carolina St.", "N.C. State")
                team = team.replace("North Carolina Asheville", "UNC Asheville")
                team = team.replace("South Carolina Upstate", "USC Upstate")
                team = team.replace("Albany NY", "Albany")
                team = team.replace("Prairie View", "Prairie View A&M")
                team = team.replace("Missouri Kansas City", "UMKC")
                team = team.replace("Long Island University", "LIU")
                team = team.replace("Massachusetts Lowell", "UMass Lowell")
                team = team.replace("Maryland Baltimore County", "UMBC")
                team = team.replace("North Carolina Wilmington", "UNC Wilmington")
                team = team.replace("Saint Francis PA", "St. Francis PA")
                team = team.replace("Grambling", "Grambling St.")
                team = team.replace("Central Connecticut St.", "Central Connecticut")
                team = team.replace("Detroit Mercy", "Detroit")
                team = team.replace("Texas A&M Corpus Christi", "Texas A&M Corpus Chris")
                team = team.replace("Omaha", "Nebraska Omaha")
                team = team.replace("Texas Arlington", "UT Arlington")
                team = team.replace("Florida International", "FIU")
                team = team.replace("California Baptist", "Cal Baptist")
                team = team.replace("Bowling Green St.", "Bowling Green")
                team = team.replace("Texas San Antonio", "UTSA")
                team = team.replace("Texas Rio Grande Valley", "UT Rio Grande Valley")
                team = team.replace("Texas El Paso", "UTEP")
                team = team.replace("Alabama Birmingham", "UAB")
                team = team.replace("Southern Mississippi", "Southern Miss")
                team = team.replace("Citadel", "The Citadel")
                team = team.replace("North Carolina Greensboro", "UNC Greensboro")
                team = team.replace("Southern Methodist", "SMU")
                team = team.replace("Southern California", "USC")
                team = team.replace("Pennsylvania", "Penn")
                team = team.replace("Nevada Las Vegas", "UNLV")
                team = team.replace("College Charleston", "Charleston")
                team = team.replace("St. John's NY", "St. John's")
                team = team.replace("Loyola IL", "Loyola Chicago")
                team = team.replace("Brigham Young", "BYU")
                team = team.replace("Virginia Commonwealth", "VCU")
                team = team.replace("Central Florida", "UCF")
                team = team.replace("Louisiana St.", "LSU")
                team = team.replace("Texas Christian", "TCU")
                team = team.replace("Saint Mary's CA", "Saint Mary's")

            teams_dict[team]['Pace'] = pace
            teams_dict[team]['Three_Par'] = Three_Par
            teams_dict[team]['True_Shoot'] = TS
            teams_dict[team]['Tov'] = Tov
    for x in opps:
        if x.find("a") == None or x.findAll("td")[16] == None:
            pass
        else:
            team = x.find('a').text
            pace = x.findAll('td')[16].text
            Three_Par = x.findAll('td')[19].text
            TS = x.findAll('td')[20].text
            Tov = x.findAll('td')[26].text

            if "Savannah St" in team:
              continue
            if "Merrimack" in team:
              continue            
            if "State" in team:
                team = team.replace("State", "St.")
            if "University" in team:
                team = team.replace("University ", "")
            if "of " in team:
                team = team.replace("of ", "")
            if "-" in team or "(" in team or ")" in team:
                team = team.replace("-", " ")
                team = team.replace("(", "")
                team = team.replace(")", "")
            if team in ["Cal St. Long Beach", "North Carolina St.", "North Carolina Asheville","South Carolina Upstate","Albany NY","Prairie View","Missouri Kansas City","Long Island University","Massachusetts Lowell","Maryland Baltimore County","North Carolina Wilmington","Saint Francis PA","Grambling","Central Connecticut St.","Detroit Mercy","Texas A&M Corpus Christi","Omaha","Texas Arlington","Florida International","California Baptist","Bowling Green St.","Texas San Antonio","Texas Rio Grande Valley","Texas El Paso","Alabama Birmingham","Southern Mississippi","Citadel","North Carolina Greensboro","Southern Methodist","Southern California","Pennsylvania","Nevada Las Vegas","College Charleston","St. John's NY","Loyola IL","Brigham Young","Virginia Commonwealth","Central Florida","Louisiana St.", "Texas Christian", "Saint Mary's CA"]:
                team = team.replace("Cal St. Long Beach", "Long Beach St.")
                team = team.replace("North Carolina St.", "N.C. State")
                team = team.replace("North Carolina Asheville", "UNC Asheville")
                team = team.replace("South Carolina Upstate", "USC Upstate")
                team = team.replace("Albany NY", "Albany")
                team = team.replace("Prairie View", "Prairie View A&M")
                team = team.replace("Missouri Kansas City", "UMKC")
                team = team.replace("Long Island University", "LIU")
                team = team.replace("Massachusetts Lowell", "UMass Lowell")
                team = team.replace("Maryland Baltimore County", "UMBC")
                team = team.replace("North Carolina Wilmington", "UNC Wilmington")
                team = team.replace("Saint Francis PA", "St. Francis PA")
                team = team.replace("Grambling", "Grambling St.")
                team = team.replace("Central Connecticut St.", "Central Connecticut")
                team = team.replace("Detroit Mercy", "Detroit")
                team = team.replace("Texas A&M Corpus Christi", "Texas A&M Corpus Chris")
                team = team.replace("Omaha", "Nebraska Omaha")
                team = team.replace("Texas Arlington", "UT Arlington")
                team = team.replace("Florida International", "FIU")
                team = team.replace("California Baptist", "Cal Baptist")
                team = team.replace("Bowling Green St.", "Bowling Green")
                team = team.replace("Texas San Antonio", "UTSA")
                team = team.replace("Texas Rio Grande Valley", "UT Rio Grande Valley")
                team = team.replace("Texas El Paso", "UTEP")
                team = team.replace("Alabama Birmingham", "UAB")
                team = team.replace("Southern Mississippi", "Southern Miss")
                team = team.replace("Citadel", "The Citadel")
                team = team.replace("North Carolina Greensboro", "UNC Greensboro")
                team = team.replace("Southern Methodist", "SMU")
                team = team.replace("Southern California", "USC")
                team = team.replace("Pennsylvania", "Penn")
                team = team.replace("Nevada Las Vegas", "UNLV")
                team = team.replace("College Charleston", "Charleston")
                team = team.replace("St. John's NY", "St. John's")
                team = team.replace("Loyola IL", "Loyola Chicago")
                team = team.replace("Brigham Young", "BYU")
                team = team.replace("Virginia Commonwealth", "VCU")
                team = team.replace("Central Florida", "UCF")
                team = team.replace("Louisiana St.", "LSU")
                team = team.replace("Texas Christian", "TCU")
                team = team.replace("Saint Mary's CA", "Saint Mary's")
            
            teams_dict[team]['Opp_Three_Par'] = Three_Par
            teams_dict[team]['Opp_True_Shoot'] = TS
            teams_dict[team]['Opp_Tov'] = Tov

def formatting(teams_dict):
    for x in teams_dict.keys():
        teams_dict[x]['record_date'] = dt.datetime.now()   
    teams_list = []
    sorted_teams_dict = sorted(teams_dict.items(), key=operator.itemgetter(0))
    for x,y in sorted_teams_dict:
        teams_list.append(y)
    return teams_list

def get_teams():
    teams = {}
    KP = 'https://kenpom.com/'
    BP = []
    for x in range(1,16):
        BP.append('http://www.espn.com/mens-college-basketball/bpi/_/view/bpi/page/' + str(x))
    SRS = "https://www.sports-reference.com/cbb/seasons/2020-ratings.html"
    
    add_ken_pom(teams, KP)
    add_BPI(teams, BP)
    add_sports_ref(teams, SRS)
    add_advanced(teams)


    teams= formatting(teams)
    return teams

In [6]:
games1 = get_yest_games()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[<div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/michigan-state/2020.html">Michigan State</a><span class="pollrank"> (1) </span></td>
<td class="right">62</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-05-21-kentucky.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/kentucky/2020.html">Kentucky</a><span class="pollrank"> (2) </span></td>
<td class="right">69</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/kansas/2020.html">Kansas</a><span class="pollrank"> (3) </span></td>
<td class="right">66</td>
<td class="right gamelink">
<a href="/cbb/boxsco

[<div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/notre-dame/2020.html">Notre Dame</a></td>
<td class="right">65</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-06-19-north-carolina.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/north-carolina/2020.html">UNC</a><span class="pollrank"> (9) </span></td>
<td class="right">76</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="winner">
<td><a href="/cbb/schools/virginia/2020.html">Virginia</a><span class="pollrank"> (11) </span></td>
<td class="right">48</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-06-21-syracuse.html">Final</a>
</td>
</tr>
<tr class="loser">
<td><a href="/cbb/schools/syracuse/2020.html">Syracuse</a></td>
<td class="right">34</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summ

[<div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="winner">
<td><a href="/cbb/schools/alabama-birmingham/2020.html">Alabama-Birmingham</a></td>
<td class="right">76</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-07-20-troy.html">Final</a>
</td>
</tr>
<tr class="loser">
<td><a href="/cbb/schools/troy/2020.html">Troy</a></td>
<td class="right">75</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/bryant/2020.html">Bryant</a></td>
<td class="right">71</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-07-19-rutgers.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/rutgers/2020.html">Rutgers</a></td>
<td class="right">73</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a>Crown Colle

[<div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/eastern-kentucky/2020.html">Eastern Kentucky</a></td>
<td class="right">49</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-08-19-kentucky.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/kentucky/2020.html">Kentucky</a><span class="pollrank"> (2) </span></td>
<td class="right">91</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/north-carolina-greensboro/2020.html">UNC Greensboro</a></td>
<td class="right">62</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-08-21-kansas.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/kansas/2020.html">Kansas</a><span class="pollrank"> (3) </span></td>
<td class="right">74</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, 

[<div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/rhode-island/2020.html">Rhode Island</a></td>
<td class="right">55</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-09-21-maryland.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/maryland/2020.html">Maryland</a><span class="pollrank"> (7) </span></td>
<td class="right">73</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/arkansas-pine-bluff/2020.html">Arkansas-Pine Bluff</a></td>
<td class="right">60</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2019-11-09-20-gonzaga.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/gonzaga/2020.html">Gonzaga</a><span class="pollrank"> (8) </span></td>
<td class="right">110</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div 

...

[<div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="winner">
<td><a href="/cbb/schools/akron/2020.html">Akron</a></td>
<td class="right">80</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2020-02-14-19-central-michigan.html">Final</a>
</td>
</tr>
<tr class="loser">
<td><a href="/cbb/schools/central-michigan/2020.html">Central Michigan</a></td>
<td class="right">67</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="winner">
<td><a href="/cbb/schools/brown/2020.html">Brown</a></td>
<td class="right">75</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2020-02-14-19-pennsylvania.html">Final</a>
</td>
</tr>
<tr class="loser">
<td><a href="/cbb/schools/pennsylvania/2020.html">Penn</a></td>
<td class="right">63</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="winner">
<t

[<div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="winner">
<td><a href="/cbb/schools/san-diego-state/2020.html">San Diego State</a><span class="pollrank"> (4) </span></td>
<td class="right">72</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2020-02-16-16-boise-state.html">Final</a>
</td>
</tr>
<tr class="loser">
<td><a href="/cbb/schools/boise-state/2020.html">Boise State</a></td>
<td class="right">55</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="winner">
<td><a href="/cbb/schools/villanova/2020.html">Villanova</a><span class="pollrank"> (15) </span></td>
<td class="right">76</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2020-02-16-13-temple.html">Final</a>
</td>
</tr>
<tr class="loser">
<td><a href="/cbb/schools/temple/2020.html">Temple</a></td>
<td class="right">56</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="g

[<div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/iowa-state/2020.html">Iowa State</a></td>
<td class="right">71</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2020-02-17-21-kansas.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/kansas/2020.html">Kansas</a><span class="pollrank"> (3) </span></td>
<td class="right">91</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/arkansas-pine-bluff/2020.html">Arkansas-Pine Bluff</a></td>
<td class="right">52</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2020-02-17-20-alcorn-state.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/alcorn-state/2020.html">Alcorn State</a></td>
<td class="right">60</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<

[<div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/duke/2020.html">Duke</a><span class="pollrank"> (6) </span></td>
<td class="right">66</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2020-02-19-21-north-carolina-state.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/north-carolina-state/2020.html">NC State</a></td>
<td class="right">88</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/syracuse/2020.html">Syracuse</a></td>
<td class="right">66</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2020-02-19-19-louisville.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/louisville/2020.html">Louisville</a><span class="pollrank"> (11) </span></td>
<td class="right">90</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class

[<div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/san-francisco/2020.html">San Francisco</a></td>
<td class="right">54</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2020-02-20-23-gonzaga.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/gonzaga/2020.html">Gonzaga</a><span class="pollrank"> (2) </span></td>
<td class="right">71</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="game_summary nohover">
<table class="teams">
<tbody>
<tr class="loser">
<td><a href="/cbb/schools/oregon/2020.html">Oregon</a><span class="pollrank"> (14) </span></td>
<td class="right">72</td>
<td class="right gamelink">
<a href="/cbb/boxscores/2020-02-20-21-arizona-state.html">Final</a>
</td>
</tr>
<tr class="winner">
<td><a href="/cbb/schools/arizona-state/2020.html">Arizona State</a></td>
<td class="right">77</td>
<td class="right"> 
			</td>
</tr>
</tbody>
</table>
</div>, <div class="ga

In [7]:
games = pd.DataFrame(games1,columns =['Date','Winner','Win_Score','Loser','Lose_Score','Home','Away'])

In [8]:
games['Winner'] = np.where(games['Winner']=="Ole Miss","Mississippi",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UCSB","UC-Santa Barbara",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UMass-Lowell","Massachusetts-Lowell",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UMass","Massachusetts",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UIC","Illinois-Chicago",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UConn","Connecticut",games['Winner'])
games['Winner'] = np.where(games['Winner']=="Saint Mary's","Saint Mary's (CA)",games['Winner'])
games['Winner'] = np.where(games['Winner']=="VCU","Virginia Commonwealth",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UMKC","Missouri-Kansas City",games['Winner'])
games['Winner'] = np.where(games['Winner']=="St. Joseph's","Saint Joseph's",games['Winner'])
games['Winner'] = np.where(games['Winner']=="BYU","Brigham Young",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UNC","North Carolina",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UTSA","Texas-San Antonio",games['Winner'])
games['Winner'] = np.where(games['Winner']=="Pitt","Pittsburgh",games['Winner'])
games['Winner'] = np.where(games['Winner']=="ETSU","East Tennessee State",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UT-Martin","Tennessee-Martin",games['Winner'])
games['Winner'] = np.where(games['Winner']=="Penn","Pennsylvania",games['Winner'])
games['Winner'] = np.where(games['Winner']=="California","University of California",games['Winner'])
games['Winner'] = np.where(games['Winner']=="USC","Southern California",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UTEP","Texas-El Paso",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UNC Asheville","North Carolina-Asheville",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UNC Greensboro","North Carolina-Greensboro",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UNC Wilmington","North Carolina-Wilmington",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UNLV","Nevada-Las Vegas",games['Winner'])
games['Winner'] = np.where(games['Winner']=="SMU","Southern Methodist",games['Winner'])
games['Winner'] = np.where(games['Winner']=="LIU","Long Island University",games['Winner'])
games['Winner'] = np.where(games['Winner']=="NC State","North Carolina State",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UCF","Central Florida",games['Winner'])
games['Winner'] = np.where(games['Winner']=="Southern Miss","Southern Mississippi",games['Winner'])
games['Winner'] = np.where(games['Winner']=="LSU","Louisiana State",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UIC","Illinois-Chicago",games['Winner'])
games['Winner'] = np.where(games['Winner']=="UMBC","Maryland-Baltimore County",games['Winner'])

In [9]:
games['Loser'] = np.where(games['Loser']=="Ole Miss","Mississippi",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UCSB","UC-Santa Barbara",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UMass-Lowell","Massachusetts-Lowell",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UMass","Massachusetts",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UIC","Illinois-Chicago",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UConn","Connecticut",games['Loser'])
games['Loser'] = np.where(games['Loser']=="Saint Mary's","Saint Mary's (CA)",games['Loser'])
games['Loser'] = np.where(games['Loser']=="VCU","Virginia Commonwealth",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UMKC","Missouri-Kansas City",games['Loser'])
games['Loser'] = np.where(games['Loser']=="St. Joseph's","Saint Joseph's",games['Loser'])
games['Loser'] = np.where(games['Loser']=="BYU","Brigham Young",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UNC","North Carolina",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UTSA","Texas-San Antonio",games['Loser'])
games['Loser'] = np.where(games['Loser']=="Pitt","Pittsburgh",games['Loser'])
games['Loser'] = np.where(games['Loser']=="ETSU","East Tennessee State",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UT-Martin","Tennessee-Martin",games['Loser'])
games['Loser'] = np.where(games['Loser']=="Penn","Pennsylvania",games['Loser'])
games['Loser'] = np.where(games['Loser']=="California","University of California",games['Loser'])
games['Loser'] = np.where(games['Loser']=="USC","Southern California",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UTEP","Texas-El Paso",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UNC Asheville","North Carolina-Asheville",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UNC Greensboro","North Carolina-Greensboro",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UNC Wilmington","North Carolina-Wilmington",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UNLV","Nevada-Las Vegas",games['Loser'])
games['Loser'] = np.where(games['Loser']=="SMU","Southern Methodist",games['Loser'])
games['Loser'] = np.where(games['Loser']=="LIU","Long Island University",games['Loser'])
games['Loser'] = np.where(games['Loser']=="NC State","North Carolina State",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UCF","Central Florida",games['Loser'])
games['Loser'] = np.where(games['Loser']=="Southern Miss","Southern Mississippi",games['Loser'])
games['Loser'] = np.where(games['Loser']=="LSU","Louisiana State",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UIC","Illinois-Chicago",games['Loser'])
games['Loser'] = np.where(games['Loser']=="UMBC","Maryland-Baltimore County",games['Loser'])

In [10]:
games['Home'] = np.where(games['Home']=="Ole Miss","Mississippi",games['Home'])
games['Home'] = np.where(games['Home']=="UCSB","UC-Santa Barbara",games['Home'])
games['Home'] = np.where(games['Home']=="UMass-Lowell","Massachusetts-Lowell",games['Home'])
games['Home'] = np.where(games['Home']=="UMass","Massachusetts",games['Home'])
games['Home'] = np.where(games['Home']=="UIC","Illinois-Chicago",games['Home'])
games['Home'] = np.where(games['Home']=="UConn","Connecticut",games['Home'])
games['Home'] = np.where(games['Home']=="Saint Mary's","Saint Mary's (CA)",games['Home'])
games['Home'] = np.where(games['Home']=="VCU","Virginia Commonwealth",games['Home'])
games['Home'] = np.where(games['Home']=="UMKC","Missouri-Kansas City",games['Home'])
games['Home'] = np.where(games['Home']=="St. Joseph's","Saint Joseph's",games['Home'])
games['Home'] = np.where(games['Home']=="BYU","Brigham Young",games['Home'])
games['Home'] = np.where(games['Home']=="UNC","North Carolina",games['Home'])
games['Home'] = np.where(games['Home']=="UTSA","Texas-San Antonio",games['Home'])
games['Home'] = np.where(games['Home']=="Pitt","Pittsburgh",games['Home'])
games['Home'] = np.where(games['Home']=="ETSU","East Tennessee State",games['Home'])
games['Home'] = np.where(games['Home']=="UT-Martin","Tennessee-Martin",games['Home'])
games['Home'] = np.where(games['Home']=="Penn","Pennsylvania",games['Home'])
games['Home'] = np.where(games['Home']=="California","University of California",games['Home'])
games['Home'] = np.where(games['Home']=="USC","Southern California",games['Home'])
games['Home'] = np.where(games['Home']=="UTEP","Texas-El Paso",games['Home'])
games['Home'] = np.where(games['Home']=="UNC Asheville","North Carolina-Asheville",games['Home'])
games['Home'] = np.where(games['Home']=="UNC Greensboro","North Carolina-Greensboro",games['Home'])
games['Home'] = np.where(games['Home']=="UNC Wilmington","North Carolina-Wilmington",games['Home'])
games['Home'] = np.where(games['Home']=="UNLV","Nevada-Las Vegas",games['Home'])
games['Home'] = np.where(games['Home']=="SMU","Southern Methodist",games['Home'])
games['Home'] = np.where(games['Home']=="LIU","Long Island University",games['Home'])
games['Home'] = np.where(games['Home']=="NC State","North Carolina State",games['Home'])
games['Home'] = np.where(games['Home']=="UCF","Central Florida",games['Home'])
games['Home'] = np.where(games['Home']=="Southern Miss","Southern Mississippi",games['Home'])
games['Home'] = np.where(games['Home']=="LSU","Louisiana State",games['Home'])
games['Home'] = np.where(games['Home']=="UIC","Illinois-Chicago",games['Home'])
games['Home'] = np.where(games['Home']=="UMBC","Maryland-Baltimore County",games['Home'])

In [11]:
games['Away'] = np.where(games['Away']=="Ole Miss","Mississippi",games['Away'])
games['Away'] = np.where(games['Away']=="UCSB","UC-Santa Barbara",games['Away'])
games['Away'] = np.where(games['Away']=="UMass-Lowell","Massachusetts-Lowell",games['Away'])
games['Away'] = np.where(games['Away']=="UMass","Massachusetts",games['Away'])
games['Away'] = np.where(games['Away']=="UIC","Illinois-Chicago",games['Away'])
games['Away'] = np.where(games['Away']=="UConn","Connecticut",games['Away'])
games['Away'] = np.where(games['Away']=="Saint Mary's","Saint Mary's (CA)",games['Away'])
games['Away'] = np.where(games['Away']=="VCU","Virginia Commonwealth",games['Away'])
games['Away'] = np.where(games['Away']=="UMKC","Missouri-Kansas City",games['Away'])
games['Away'] = np.where(games['Away']=="St. Joseph's","Saint Joseph's",games['Away'])
games['Away'] = np.where(games['Away']=="BYU","Brigham Young",games['Away'])
games['Away'] = np.where(games['Away']=="UNC","North Carolina",games['Away'])
games['Away'] = np.where(games['Away']=="UTSA","Texas-San Antonio",games['Away'])
games['Away'] = np.where(games['Away']=="Pitt","Pittsburgh",games['Away'])
games['Away'] = np.where(games['Away']=="ETSU","East Tennessee State",games['Away'])
games['Away'] = np.where(games['Away']=="UT-Martin","Tennessee-Martin",games['Away'])
games['Away'] = np.where(games['Away']=="Penn","Pennsylvania",games['Away'])
games['Away'] = np.where(games['Away']=="California","University of California",games['Away'])
games['Away'] = np.where(games['Away']=="USC","Southern California",games['Away'])
games['Away'] = np.where(games['Away']=="UTEP","Texas-El Paso",games['Away'])
games['Away'] = np.where(games['Away']=="UNC Asheville","North Carolina-Asheville",games['Away'])
games['Away'] = np.where(games['Away']=="UNC Greensboro","North Carolina-Greensboro",games['Away'])
games['Away'] = np.where(games['Away']=="UNC Wilmington","North Carolina-Wilmington",games['Away'])
games['Away'] = np.where(games['Away']=="UNLV","Nevada-Las Vegas",games['Away'])
games['Away'] = np.where(games['Away']=="SMU","Southern Methodist",games['Away'])
games['Away'] = np.where(games['Away']=="LIU","Long Island University",games['Away'])
games['Away'] = np.where(games['Away']=="NC State","North Carolina State",games['Away'])
games['Away'] = np.where(games['Away']=="UCF","Central Florida",games['Away'])
games['Away'] = np.where(games['Away']=="Southern Miss","Southern Mississippi",games['Away'])
games['Away'] = np.where(games['Away']=="LSU","Louisiana State",games['Away'])
games['Away'] = np.where(games['Away']=="UIC","Illinois-Chicago",games['Away'])
games['Away'] = np.where(games['Away']=="UMBC","Maryland-Baltimore County",games['Away'])

In [12]:
games = games.drop(['Win_Score','Lose_Score'],axis=1)
games['WinnerHome'] = np.where(games['Winner']==games['Home'],1,0)

In [13]:
statlines = pd.read_csv("statlines-2-11.csv", parse_dates=['record_date'])
statlines['Date'] = statlines['record_date'].dt.date
statlines = statlines[['Date','sr_name','KenPomVal','BPIval','SRS']]
statlines = statlines[statlines['Date'] >= dt.date(year=2019,month=11,day=18)]
statlines = statlines.drop_duplicates()

In [14]:
latest_stats = get_teams()
latest_stats = pd.DataFrame(latest_stats)
latest_stats = latest_stats[['Team','KenPomVal','BPIval','SRS']]

In [15]:
latest_stats

,Team,KenPomVal,BPIval,SRS
0,Abilene Christian,-0.61,0.7,-2.54
1,Air Force,-2.74,-1.4,-0.70
2,Akron,+10.91,6.9,7.81
3,Alabama,+12.69,7.7,11.93
4,Alabama A&M,-19.05,-12.9,-18.30
...,...,...,...,...
347,Wright State,+3.90,3.0,3.04
348,Wyoming,-10.40,-8.3,-5.04
349,Xavier,+14.44,8.7,12.65
350,Yale,+13.60,9.0,8.26


## Make table with Latest Stats. Then Make table with day of stats. Concat two and delete duplicates only keeping the second

In [16]:
half_join_latest = games.merge(latest_stats,left_on=['Away'],right_on=['Team'],how='left')
full_join_latest = half_join_latest.merge(latest_stats,left_on=['Home'],right_on=['Team'],how='left',suffixes=('_Away','_Home'))
full_join_latest = full_join_latest.drop(['Winner','Loser','Team_Home','Team_Away'],axis = 1)

In [17]:
half_join_day_of = games.merge(statlines,left_on=['Date','Away'],right_on=['Date','sr_name'],how='inner')
full_join_day_of = half_join_day_of.merge(statlines,left_on=['Date','Home'],right_on=['Date','sr_name'],how='inner',suffixes=('_Away','_Home'))
full_join_day_of = full_join_day_of.drop(['Winner','Loser','sr_name_Home','sr_name_Away'],axis = 1)

In [43]:
full_join_day_of

,Date,Home,Away,WinnerHome,KenPomVal_Away,BPIval_Away,SRS_Away,KenPomVal_Home,BPIval_Home,SRS_Home
0,2019-11-18,Michigan State,Charleston Southern,1,-9.21,-3.0,-27.87,29.08,12.4,26.03
1,2019-11-18,Kentucky,Utah Valley,1,-2.64,-2.8,-8.70,26.31,9.7,34.27
2,2019-11-18,Ohio State,Stetson,1,-15.39,-5.7,-28.72,23.00,8.1,28.78
3,2019-11-18,Utah State,Texas-San Antonio,1,-3.21,-0.7,-9.54,13.44,6.8,27.04
4,2019-11-18,Auburn,Colgate,1,3.31,0.6,-5.60,17.98,8.5,9.76
...,...,...,...,...,...,...,...,...,...,...
2372,2020-02-10,Florida A&M,North Carolina A&T,1,-9.47,-6.4,-9.10,-12.43,-8.9,-12.03
2373,2020-02-10,Morgan State,North Carolina Central,0,-8.29,-5.5,-9.99,-13.52,-8.1,-12.59
2374,2020-02-10,Northern Colorado,Portland State,0,-2.77,-1.2,-3.65,9.38,6.9,5.05
2375,2020-02-10,Winthrop,Radford,0,-0.79,1.2,-2.51,5.73,3.3,3.15


In [44]:
full = full_join_latest.append(full_join_day_of)
full = full.drop_duplicates(subset=['Date', 'Home'],keep='last')
full = full.dropna()

In [45]:
full

,Date,Home,Away,WinnerHome,KenPomVal_Away,BPIval_Away,SRS_Away,KenPomVal_Home,BPIval_Home,SRS_Home
0,2019-11-05,Kentucky,Michigan State,1,+22.28,15.9,21.23,+17.73,11.7,15.73
1,2019-11-05,Duke,Kansas,1,+31.01,18.4,25.79,+25.70,18.4,23.54
2,2019-11-05,Miami (FL),Louisville,0,+21.98,14.7,18.26,+6.81,4.8,6.79
3,2019-11-05,Florida,North Florida,1,+0.90,1.2,-0.34,+15.93,11.5,14.50
4,2019-11-05,Maryland,Holy Cross,1,-20.72,-13.3,-16.92,+22.49,13.8,18.59
...,...,...,...,...,...,...,...,...,...,...
2372,2020-02-10,Florida A&M,North Carolina A&T,1,-9.47,-6.4,-9.1,-12.43,-8.9,-12.03
2373,2020-02-10,Morgan State,North Carolina Central,0,-8.29,-5.5,-9.99,-13.52,-8.1,-12.59
2374,2020-02-10,Northern Colorado,Portland State,0,-2.77,-1.2,-3.65,9.38,6.9,5.05
2375,2020-02-10,Winthrop,Radford,0,-0.79,1.2,-2.51,5.73,3.3,3.15


In [46]:
full.to_csv("year19-20.csv")

In [21]:
full[full['Home']=='TCU']

,Date,Home,Away,WinnerHome,KenPomVal_Away,BPIval_Away,SRS_Away,KenPomVal_Home,BPIval_Home,SRS_Home


In [22]:
X = full.iloc[:,[4,5,6,7,8,9]]
y = full.iloc[:,3]

In [23]:
y

0       1
1       1
2       0
3       1
4       1
       ..
2372    1
2373    0
2374    0
2375    0
2376    1
Name: WinnerHome, Length: 4232, dtype: int64

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)

In [25]:
LR = LogisticRegression().fit(X_train, y_train)

In [26]:
LR.score(X_test, y_test)

0.7461629279811098

In [27]:
LR.coef_ , LR.intercept_, LR.n_iter_

(array([[-0.04788313, -0.08342733, -0.01993755,  0.1027388 ,  0.00619907,
          0.01709052]]),
 array([0.50084562]),
 array([29], dtype=int32))

In [28]:
from sklearn import metrics
y_pred = LR.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[153, 135],
       [ 80, 479]])

In [29]:
# import pickle
# filename = 'LogR2-11.sav'
# pickle.dump(LR, open(filename, 'wb'))

# Function to determine confidence of game

In [30]:
def conf(game):
    arr = LR.coef_
    print("Coefficients:",arr)
    print("Team Stats:",game)
    coefmult = arr*game
    print("Multiplied:",coefmult)
    comb_coef = coefmult.sum()+LR.intercept_
    print("Sum of multiplied:",comb_coef)
    es = math.exp(comb_coef)
    print("E to the sum of multiplied:",es)
    confidence = es / (1+es)
    print("conf:", confidence)

In [31]:
def confnuet(game):
    arr = np.array([[-0.04160335,-0.08892401,-0.03608732,0.04160335,0.08892401,0.03608732]])#LR.coef_
    print("Coefficients:",arr)
    print("Team Stats:",game)
    coefmult = arr*game
    print("Multiplied:",coefmult)
    comb_coef = coefmult.sum()#+LR.intercept_
    print("Sum of multiplied:",comb_coef)
    es = math.exp(comb_coef)
    print("E to the sum of multiplied:",es)
    confidence = es / (1+es)
    print("Nuetral Confi:", confidence)

In [32]:
KUatDuke = [31.05,18.3,25.85,27.64,19.2,25.01]
predict = [KUatDuke]
res = LR.predict_proba(predict)
confidence = confnuet(KUatDuke)
print()
print("Prob from Model",res)


Coefficients: [[-0.04160335 -0.08892401 -0.03608732  0.04160335  0.08892401  0.03608732]]
Team Stats: [31.05, 18.3, 25.85, 27.64, 19.2, 25.01]
Multiplied: [[-1.29178402 -1.62730938 -0.93285722  1.14991659  1.70734099  0.90254387]]
Sum of multiplied: -0.09214916329999989
E to the sum of multiplied: 0.9119691070775613
Nuetral Confi: 0.4769789970464027

Prob from Model [[0.4114134 0.5885866]]


In [33]:
DukeAtKU = [27.64,19.2,25.01,31.05,18.3,25.85]
predict = [DukeAtKU]
res = LR.predict_proba(predict)
confidence = confnuet(DukeAtKU)
print()
print("Prob from Model",res)
print(sum(sum(res)))

Coefficients: [[-0.04160335 -0.08892401 -0.03608732  0.04160335  0.08892401  0.03608732]]
Team Stats: [27.64, 19.2, 25.01, 31.05, 18.3, 25.85]
Multiplied: [[-1.14991659 -1.70734099 -0.90254387  1.29178402  1.62730938  0.93285722]]
Sum of multiplied: 0.09214916329999978
E to the sum of multiplied: 1.0965283716731775
Nuetral Confi: 0.5230210029535972

Prob from Model [[0.3052998 0.6947002]]
1.0


# Home KP,Home BPI, Home SRS, Same for Away

## 1 means Home Team should win

In [34]:
UKatLSU = [17.39,11.5,15.67,15.77,11.1,14.44]
predict = [UKatLSU]
res = LR.predict_proba(predict)
confidence = conf(UKatLSU)
print()
print(res)
print(sum(sum(res)))

Coefficients: [[-0.04788313 -0.08342733 -0.01993755  0.1027388   0.00619907  0.01709052]]
Team Stats: [17.39, 11.5, 15.67, 15.77, 11.1, 14.44]
Multiplied: [[-0.83268763 -0.95941424 -0.31242136  1.62019095  0.06880971  0.24678713]]
Sum of multiplied: [0.33211018]
E to the sum of multiplied: 1.3939064214451267
conf: 0.5822727275210987

[[0.41772727 0.58227273]]
1.0


In [35]:
MichAtRutg = [20.48,12.4,18.52,17.61,10.5,15.8]
predict = [MichAtRutg]
res = LR.predict_proba(predict)
confidence = conf(MichAtRutg)
print()
print(res)

Coefficients: [[-0.04788313 -0.08342733 -0.01993755  0.1027388   0.00619907  0.01709052]]
Team Stats: [20.48, 12.4, 18.52, 17.61, 10.5, 15.8]
Multiplied: [[-0.9806465  -1.03449883 -0.36924337  1.80923035  0.06509027  0.27003024]]
Sum of multiplied: [0.26080777]
E to the sum of multiplied: 1.2979781357559916
conf: 0.5648348500622183

[[0.43516515 0.56483485]]


In [36]:
OreAtASU = [18.8,12.4,17.56,12.01,7.3,11.75]
predict = [OreAtASU]
res = LR.predict_proba(predict)
confidence = conf(OreAtASU)
print()
print(res)

Coefficients: [[-0.04788313 -0.08342733 -0.01993755  0.1027388   0.00619907  0.01709052]]
Team Stats: [18.8, 12.4, 17.56, 12.01, 7.3, 11.75]
Multiplied: [[-0.90020284 -1.03449883 -0.35010333  1.23389305  0.04525323  0.20081363]]
Sum of multiplied: [-0.30399948]
E to the sum of multiplied: 0.737861252825507
conf: 0.4245800702592644

[[0.57541993 0.42458007]]


In [37]:
KUatBaylor = [30.93,18.2,25.64,26.82,15.4,20.58]
predict = [KUatBaylor]
res = LR.predict_proba(predict)
confidence = conf(KUatBaylor)
print()
print(res)

Coefficients: [[-0.04788313 -0.08342733 -0.01993755  0.1027388   0.00619907  0.01709052]]
Team Stats: [30.93, 18.2, 25.64, 26.82, 15.4, 20.58]
Multiplied: [[-1.48102521 -1.51837732 -0.51119871  2.75545474  0.09546572  0.35172294]]
Sum of multiplied: [0.19288779]
E to the sum of multiplied: 1.2127467007258113
conf: 0.5480729901565388

[[0.45192701 0.54807299]]


In [38]:
SMUatTulsa = [10.55,6.6,9.34,9.71,5.8,8.09]
predict = [SMUatTulsa]
res = LR.predict_proba(predict)
confidence = conf(SMUatTulsa)
print()
print(res)

Coefficients: [[-0.04788313 -0.08342733 -0.01993755  0.1027388   0.00619907  0.01709052]]
Team Stats: [10.55, 6.6, 9.34, 9.71, 5.8, 8.09]
Multiplied: [[-0.50516702 -0.55062035 -0.18621669  0.99759379  0.03595462  0.13826232]]
Sum of multiplied: [0.4306523]
E to the sum of multiplied: 1.538260602661121
conf: 0.6060294207176375

[[0.39397058 0.60602942]]


In [39]:
HofAtDel = [4.51,3.2,2.17,-1.51,-.4,-2.08]
predict = [HofAtDel]
res = LR.predict_proba(predict)
confidence = conf(HofAtDel)
print()
print(res)

Coefficients: [[-0.04788313 -0.08342733 -0.01993755  0.1027388   0.00619907  0.01709052]]
Team Stats: [4.51, 3.2, 2.17, -1.51, -0.4, -2.08]
Multiplied: [[-0.21595292 -0.26696744 -0.04326448 -0.1551356  -0.00247963 -0.03554829]]
Sum of multiplied: [-0.21850273]
E to the sum of multiplied: 0.8037212888992112
conf: 0.44559062081576495

[[0.55440938 0.44559062]]


In [40]:
MaryAtOSU = [22.86,14,18.72,22.07,13.9,19.31]
predict = [MaryAtOSU]
res = LR.predict_proba(predict)
confidence = conf(MaryAtOSU)
print()
print(res)

Coefficients: [[-0.04788313 -0.08342733 -0.01993755  0.1027388   0.00619907  0.01709052]]
Team Stats: [22.86, 14, 18.72, 22.07, 13.9, 19.31]
Multiplied: [[-1.09460835 -1.16798255 -0.37323088  2.26744542  0.08616712  0.33001797]]
Sum of multiplied: [0.54865434]
E to the sum of multiplied: 1.7309222231760957
conf: 0.6338233320914618

[[0.36617667 0.63382333]]


In [41]:
LouisVsFSU = [21.23,14.1,17.56,19.18,12.5,16.75]
predict = [LouisVsFSU]
res = LR.predict_proba(predict)
confidence = conf(LouisVsFSU)
print()
print(res)

Coefficients: [[-0.04788313 -0.08342733 -0.01993755  0.1027388   0.00619907  0.01709052]]
Team Stats: [21.23, 14.1, 17.56, 19.18, 12.5, 16.75]
Multiplied: [[-1.01655885 -1.17632529 -0.35010333  1.97053028  0.07748841  0.28626624]]
Sum of multiplied: [0.29214308]
E to the sum of multiplied: 1.339294635157582
conf: 0.5725207141627814

[[0.42747929 0.57252071]]
